In [24]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from numpy  import array
from scipy import stats
from gensim.models.callbacks import CallbackAny2Vec
from sklearn import preprocessing

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# PREPROCESSING

In [3]:
corpus_filename = '../data/extracted_emoji_sequences.txt'

In [4]:
corpus = open(corpus_filename).read().splitlines()


In [5]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

In [6]:
def onlyEmojiSequences(tokens):
    threshold_emojis = [x for x in tokens if len(x) > 1]
    return threshold_emojis

In [7]:
tokenized_corpus = tokenize_corpus(corpus)
emojiSequences = onlyEmojiSequences(tokenized_corpus)

In [8]:
token_count = sum([len(sentence) for sentence in emojiSequences])
print("The corpus contains {0:,} emoji points.".format(token_count))

The corpus contains 610,256 emoji points.


# SETTING VARIABLES

In [9]:
# Dimensionality of the resulting word vectors.
num_features = 300

# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
num_workers = multiprocessing.cpu_count()
# num_workers = 1


# Context window length.
context_size = 4

# Downsample setting for frequent words.
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
# remove later
seed = 1

emoji2vec = w2v.Word2Vec(
    sg=1,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
        self.tempLoss = 0
    def on_epoch_end(self, model):
        print("Epoch #{}  Loss-Value: {}".format(self.epoch, (emoji2vec.get_latest_training_loss() - self.tempLoss)))
        self.epoch += 1
        self.tempLoss = emoji2vec.get_latest_training_loss()
        
epoch_logger = EpochLogger()
emoji2vec.build_vocab(emojiSequences, progress_per=2)

# TRAINING

In [10]:
emoji2vec.train(emojiSequences, total_examples=emoji2vec.corpus_count, epochs = 10, compute_loss=True, callbacks=[epoch_logger])

Epoch #0  Loss-Value: 134039.15625
Epoch #1  Loss-Value: 84423.078125
Epoch #2  Loss-Value: 77855.828125
Epoch #3  Loss-Value: 97973.15625
Epoch #4  Loss-Value: 76992.34375
Epoch #5  Loss-Value: 103297.3125
Epoch #6  Loss-Value: 83659.125
Epoch #7  Loss-Value: 82123.8125
Epoch #8  Loss-Value: 80021.3125
Epoch #9  Loss-Value: 93357.875


(3887227, 6102560)

In [11]:
if not os.path.exists("trained"):
    os.makedirs("trained")
emoji2vec.save(os.path.join("trained", "canIseed.w2v"))

# EVALUATING TRAINED MODEL

In [12]:
emoji2vec = w2v.Word2Vec.load(os.path.join("trained", "canIseed.w2v"))

In [36]:
# extracts the 508 Annotator Results as the Gold-Standard
corpus_filename = '../data/EmoSim508.json'
corpus = open(corpus_filename).read()
annotator_similarity_score_508 = list(array(re.findall('(?<=_Annotator_Agreement": )(.*?)(?=\})', corpus)))

# glyph_pairs_1016
unicode_pairs_1016 = re.findall('(?<=unicodelong": "\\\)(.*?)(?=")', corpus)    
glyph_pairs_1016 = [codecs.decode(unicode_pairs_1016[x].replace(str('\\\\'),str('\\')).replace('_',''), 'unicode_escape') for x in range(len(unicode_pairs_1016))]

# computation of Cosine Similarity
goldstandard = []
selftrained = []
for x in range(len(annotator_similarity_score_508)):
    cosineSimilarity = None
    
    emoji1 = glyph_pairs_1016.pop(0)
    emoji2 = glyph_pairs_1016.pop(0)
    
    try:
        cosineSimilarity = emoji2vec.wv.similarity(emoji1, emoji2)
    except:
        print('the cosine similarity between ' + emoji1 + ' and ' + emoji2 + ' could not be computed.')
    
    if(cosineSimilarity is not None):
        goldstandard.append(annotator_similarity_score_508.pop(0))
        selftrained.append(cosineSimilarity)

# skalierter GoldStandard
min_max_scaler = preprocessing.MinMaxScaler()
scaled_goldstandard = min_max_scaler.fit_transform(np.asarray(goldstandard).reshape(-1, 1))

print()

# computation of SPEARRANK CORRELATION COEFFICIENT
spearmanRank = stats.spearmanr(goldstandard, selftrained)
print('Der Spearman Rank Correlation Coefficient ist {}'.format(spearmanRank))

# computation of MAE
mae = mean_absolute_error(scaled_goldstandard, selftrained)
print('Der MAE ist {}'.format(mae))

# computation of MSE
mse = mean_squared_error(scaled_goldstandard, selftrained)
print('Der MSE ist {}'.format(mse))



the cosine similarity between 🇬🇧 and 🇺🇸 could not be computed.
the cosine similarity between 🏅 and 🇺🇸 could not be computed.
the cosine similarity between 🇺🇸 and ❤ could not be computed.
the cosine similarity between 🇺🇸 and 💥 could not be computed.
the cosine similarity between 🎤 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 📲 could not be computed.
the cosine similarity between 👇 and 🇳🇬 could not be computed.
the cosine similarity between 🎧 and 🇳🇬 could not be computed.
the cosine similarity between 🇳🇬 and 🎶 could not be computed.
the cosine similarity between 👏 and ↪ could not be computed.

Der Spearman Rank Correlation Coefficient ist SpearmanrResult(correlation=0.545629378093478, pvalue=5.635334943656316e-40)
Der MAE ist 0.20177384780712393
Der MSE ist 0.06207927692177068


/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U4 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [ ]:
# emoji2vec.wv.get_vector('🙁')
# emoji2vec.wv.similarity('🙁', '🤒')
# emoji2vec.wv.most_similar('💦')

In [22]:
scaled_goldstandard

array([[1.    ],
       [0.9875],
       [0.975 ],
       [0.9625],
       [0.9625],
       [0.9625],
       [0.9625],
       [0.95  ],
       [0.95  ],
       [0.95  ],
       [0.95  ],
       [0.95  ],
       [0.9375],
       [0.9375],
       [0.9375],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.925 ],
       [0.9125],
       [0.9   ],
       [0.8875],
       [0.8875],
       [0.8875],
       [0.8875],
       [0.875 ],
       [0.875 ],
       [0.8625],
       [0.8625],
       [0.85  ],
       [0.85  ],
       [0.85  ],
       [0.85  ],
       [0.85  ],
       [0.8375],
       [0.8375],
       [0.8375],
       [0.8375],
       [0.8375],
       [0.825 ],
       [0.825 ],
       [0.8125],
       [0.8   ],
       [0.8   ],
       [0.7875],
       [0.7875],
       [0.775 ],
       [0.775 ],
       [0.775 ],
       [0.775 ],
       [0.7625],
       [0.7625],
       [0.7625